This tutorial shows how to use Alpaga’s file management functions to efficiently organize and access spectral data.

In [16]:
import numpy as np
import os 
import sys

import Alpaga
from Alpaga import file_management
from Alpaga.Data_tutorial import get_tutorial_path 


# Introduction (please read carefully)

In this tutorial, we will explore the various functions available to read data obtained from our experimental device. The **main idea** is that one obtains experimentally a series of **iterations** as a function of a "parameter," which we will call **angle** — historically, this represents the light polarization. The goal of this tutorial is to explain how to connect these files to Alpaga for analysis.

An iteration is a series of acquisitions that sample the same spectrum. We aim to average the spectra across iterations for a given angle.
Practically speaking, if you record one spectrum over 10 seconds for each angle, you will have 1 iteration per angle. If you record 10 spectra over 1 second for each angle, you will have 10 iterations per angle.

If you record spectra as a function of angle — for example, from 0° to 180° in steps of 10° — you will have 19 angles. With 10 iterations per angle, you will obtain 19 × 10 = 190 spectra to process
Again, we would like to emphasize that angle can represent something other than an angle. Technically speaking, angles are treated as strings in Alpaga. 

We will show how to do this in the spectra_analysis tutorial, but first, let us discuss how to access the data — **your** data. In fact, this is likely the part where you will need to spend the most time during your first use of Alpaga.

First, let us show how this is done in the lab where Alpaga was developed.
The LabView software controlling the acquisition saves the spectra in a directory with the following naming convention:
```python
prefix + "_" + angle value + "_" + iteration number + ".dat"
```

For example, for an angle of 0.0° with 3 acquisitions, we would have 3 files:
```polarP_0.0_1.dat```, ```polarP_0.0_2.dat```, and ```polarP_0.0_3.dat```.


Given a prefix, ```angle_value```, and ```iteration```, the function `alpaga.file_management.standard_file_name` constructs the file name according to the rule presented above.

We can use `standard_file_name` to generate file names in all possible cases:

- When both angle and iteration are specified  
- When only the iteration is specified  
- When only the angle is specified  
- When neither is specified


In [17]:
# Example 1: both angle and iteration are provided
name_out = Alpaga.file_management.standard_file_name('prefixe', angle='42.0', iteration='4', extention='.dat')
print(name_out)

# Example 2: no angle provided, only iteration
name_out = Alpaga.file_management.standard_file_name('prefixe', angle=False, iteration='4', extention='.dat')
print(name_out)

# Example 3: only angle provided, no iteration
name_out = Alpaga.file_management.standard_file_name('prefixe', angle='42.0', iteration=False, extention='.dat')
print(name_out)

# Example 4: neither angle nor iteration provided
name_out = Alpaga.file_management.standard_file_name('prefixe', angle=False, iteration=False, extention='.dat')
print(name_out)

prefixe_42.0_4.dat
prefixe_4.dat
prefixe_42.0.dat
prefixe.dat


The entire Alpaga analysis framework is built around the concepts of **angle** (for the "physical" analyses) and **iteration** (for denoising and computing experimental errors.

Therefore, you may be using this exact naming convention for your files, a very similar one, or a different one entirely.
  + If you are using this convention, continue to the next section — you will likely have little difficulty following the subsequent tutorials for your analyses.
  + If not, skip to the last section.
  
In any case, Alpaga requires a way to access the files and associate each one with a specific **angle** and **iteration**.

**Note:** If you are saving your iterations sequentially for a single angle, please contact us. This case is not yet handled, but it should be easily fixed.

# About the Tutorial Data

The data used in the tutorials are included within the Alpaga package. You can access them using the `Alpaga.Data_tutorial.get_tutorial_path function`. This is already done in the tutorials, so you should not need to worry about this step.

If you want to explore additional data or access the files manually, check the `Alpaga/Data_tutorial` folder in the GitHub repository (or your local copy), or use the following command to see the directory on your system:

In [18]:
print('Tutorial data is stored here:', get_tutorial_path(''))

Tutorial data is stored here: /home/glebreton/.local/lib/python3.7/site-packages/Alpaga/Data_tutorial/


# "Standard" Naming Convention

If you are reading this section, your file naming convention follows a pattern like:
```python
prefix + "_" + angle value + "_" + iteration number + extension
```
and all files are saved in the same directory for a given experiment.

If your convention is slightly different, read this section and the next one. If you have recorded only one type of experiment (i.e., using a single prefix), you can use either the function `Alpaga.file_management.find_file_iter_from_dir` or `Alpaga.file_management.find_angle_iter_from_dir` to automatically determine the number of iterations and angles.

The following examples are self-explanatory:

In [19]:
directory = get_tutorial_path("SHS/Single_angle")
prefix_file, N_iter, extention = Alpaga.file_management.find_file_iter_from_dir(directory)
print('The prefix for all the file are: "' + prefix_file + '" with ' + str(N_iter) + ' iter. The extention is: ' + extention)

I will look at file with the extention .dat in the directory /home/glebreton/.local/lib/python3.7/site-packages/Alpaga/Data_tutorial/SHS/Single_angle for single acquisition. The type of the files should be: prefix_iter.extention
The prefix for all the file are: "Spectre_4.0" with 12 iter. The extention is: .dat


In [20]:
directory = get_tutorial_path("SHS/Eau_polar_V")
prefix_file, L_files_angles, N_iter, extention = Alpaga.file_management.find_angle_iter_from_dir(directory)
print('The prefix for all the file are: "' + prefix_file + '" with ' + str(N_iter) + ' iter. The angle found are ' + str(L_files_angles) + '. The extention is: ' + extention)

I will look at file with the extention .dat in the directory /home/glebreton/.local/lib/python3.7/site-packages/Alpaga/Data_tutorial/SHS/Eau_polar_Vfor angle dependent values. The type of the files should be: prefix_angle_iter.extention
The prefix for all the file are: "Spectre" with 12 iter. The angle found are ['4.0', '8.0', '12.0', '16.0', '20.0', '24.0', '28.0', '32.0', '36.0', '40.0', '44.0', '48.0', '52.0', '56.0', '60.0', '64.0', '68.0', '72.0', '76.0', '80.0', '84.0', '88.0', '92.0', '96.0', '100.0', '104.0', '108.0', '112.0', '116.0', '120.0', '124.0', '128.0', '132.0', '136.0', '140.0', '144.0', '148.0', '152.0', '156.0', '160.0', '164.0', '168.0', '172.0', '176.0', '180.0', '184.0', '188.0']. The extention is: .dat


The function ``find_angle_iter_from_dir(directory)`` scans the specified directory for all spectral files and organizes them according to the angle of acquisition and iteration number.

It returns four values:

- ``prefix_file``: the common prefix shared by all files in this dataset.
- ``L_files_angles``: a list of lists containing the files grouped by angle.
- ``N_iter``: the number of iterations per angle.
- ``extension``: the file extension of the dataset files.

This organization allows subsequent functions to process the spectra automatically for each angle and iteration. Using these functions, you can directly obtain the lists of "angles" and "iterations" if your naming convention matches ours.

If your naming convention is different, you can still refer to these functions to build your own solution. These functions are not mandatory for using Alpaga.

# "Slightly Different" Naming Convention

Suppose you are using a slightly different naming convention, for example: `prefix-$anglevalue-$iteration.extension or prefix_angleval$anglevalue_aq$iteration.dat`.

In this case, you will not be able to directly use the functions defined in `Alpaga.file_management` and will need to provide equivalent functions.

To stay close to the original Alpaga approach, you need to define a "convention" function and a "finder" function to replace `Alpaga.file_management.standard_file_name` and `Alpaga.file_management.find_angle_iter_from_dir`, respectively.

For example, for the "convention" function:

In [21]:
def my_own_convention_for_filename(prefixe, angle=False, iteration=False, extention='.dat'):
    """
    Define how to build the name of a file. 
    Note: The function structure, arguments, and output must match this exactly!
    """
    if isinstance(angle, bool):  # Case where no angle is given
        if isinstance(iteration, bool):  # Case where no iteration is given
            name = prefixe + extention
        else:  # Case where only an iteration is given
            name = prefixe + '_aq' + iteration + extention
    else:  # Case where an angle is given
        if isinstance(iteration, bool):  # Case where no iteration is given
            name = prefixe + '_' + angle + extention
        else:  # Case where both angle and iteration are given
            name = prefixe + '_' + angle + '_aq' + iteration + extention
    return name
    
# Example usage
name_out = my_own_convention_for_filename('prefixe', angle='42.0', iteration='4', extention='.dat')
print(name_out)


prefixe_42.0_aq4.dat


We recommend creating a function that automatically returns a list of angles and the number of iterations per acquisition by simply knowing the directory name. This is optional, however.

If you do not create this function, you will need to provide:

 + `prefix`
 + the number of iterations, `N_iter`
 + the list of angles, `L_angle`
 + the file `extension`
 + a method to map a prefix, angle value, and iteration number to a filename

**Note:** The prefix can be an empty string if necessary.

If you are using a naming convention very similar to the original Alpaga format, we recommend defining your own `my_own_convention_for_filename`. This will allow you to follow the next tutorial (`tutorial_spectra_analysis`) without additional modifications.

In [22]:
# Try out:
def my_own_convention_for_filename(prefixe, angle=False, iteration=False, extention='.dat'):
    '''
    Define how to build the name of a file. 
    Please note that the function structure / argument / output should be exactly like that!!!!
    '''
    if isinstance(angle, bool):  
        if isinstance(iteration, bool):  
            name = prefixe + extention # CHANGE THIS PART 
        else: 
            name = prefixe + '_aq' + iteration + extention
    else: 
        if isinstance(iteration, bool):  # CHANGE THIS PART 
            name = prefixe + '_' + angle + extention
        else:  
            name = prefixe + '_' + angle + '_aq' + iteration + extention # CHANGE THIS PART 
    return(name)
    
name_out = my_own_convention_for_filename('prefixe', angle='42.0', iteration='4', extention='.dat')
print(name_out)

prefixe_42.0_aq4.dat


# Different Naming Conventions

If you are using an entirely different way of naming or saving your spectra, you must provide the full path of each acquisition as an `N_angle x N_iter` list. For example, for 2 angles and 3 iterations:

```python
    L_filename = [
        [filename_angle1_iter1, filename_angle1_iter2, filename_angle1_iter3],
        [filename_angle2_iter1, filename_angle2_iter2, filename_angle2_iter3]
    ]
```

With the correct `N_iter` and any list of angles:

```python

    L_files_angles = ['angle_value_1', 'angle_value_2']
    N_iter = 2
```


**Note:** If you provide your data in this way, be careful when following the next tutorial, especially during the denoising step (Part I) and when using the `file` argument in the core function `Alpaga.analyze_run.polarisation_intensity` (Part III).
